# Evaluates RAG pipeline responses

In [3]:
import re
from openai import OpenAI
import pandas as pd
from tqdm.auto import tqdm

tqdm.pandas()  # load tqdm's pandas support
pd.set_option("display.max_colwidth", None)


In [10]:
OPENAI_API_KEY = ""

client = OpenAI(api_key=OPENAI_API_KEY)

In [11]:

JUDGE_PROMPT = """
You will be given a user_query, rag_answer and gold_answer triples.
Your task is to provide a 'total rating' scoring how well the rag_answer answers the user concerns expressed in the user_query, based on the gold_answer (validated gold standard answer).
Give your rating on a scale of 1 to 4, where 1 means that the rag_answer is not helpful at all, and 4 means that the rag_answer completely and helpfully addresses the user_query.

Here is the scale you should use to build your answer:
1: The rag_answer is terrible: completely irrelevant to the question asked, or very partial
2: The rag_answer is mostly not helpful: misses some key aspects of the question
3: The rag_answer is mostly helpful: provides support, but still could be improved
4: The rag_answer is excellent: relevant, direct, detailed, and addresses all the concerns raised in the question

Provide your feedback as follows:

Feedback:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 4)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here are the question and answer.

user_query: {user_query}
gold_answer: {gold_answer}
rag_answer: {rag_answer}

Provide your feedback. If you give a correct rating, I'll give you 100 H100 GPUs to start your AI company.
Feedback:::
Evaluation: """

In [5]:
eval_df = pd.read_csv("./data/memento_eval_qa_FZ.csv")

In [9]:
eval_sample = eval_df[["question", "answer"]].sample(5)
eval_sample

,question,answer
20,Was sind Familienzulagen in der Landwirtschaft,Das Bundesgesetz über die Familienzulagen in der Landwirtschaft (FLG) gilt\nfür in der Landwirtschaft tätige Personen. Verschiedene Bestimmungen\ndes Bundesgesetzes über die Familienzulagen (FamZG) sind jedoch auf das\nFLG anwendbar.\nWeitere Informationen zum FamZG finden Sie im Merkblatt\n6.08 – Familienzulagen.\nAnspruchsberechtigt nach dem FLG sind selbständige Landwirtinnen und\nLandwirte und Arbeitnehmende in der Landwirtschaft.\nDie Leistungen entsprechen den Mindestleistungen nach FamZG:\n• Kinderzulage von 200 Franken pro Kind und Monat\n(im Berggebiet liegen die Ansätze 20 Franken höher);\n• Ausbildungszulage von 250 Franken pro Kind und Monat\n(im Berggebiet liegen die Ansätze 20 Franken höher);\n• Haushaltungszulage von 100 Franken pro Monat für landwirtschaft-\nliche Arbeitnehmende.\nDieses Merkblatt informiert selbständige Landwirtinnen und Landwirte\nund Arbeitnehmende in der Landwirtschaft über den Anspruch auf Fami-\nlienzulagen.
24,Was ist unter «Ausbildung» zu verstehen?,"Anspruch auf Ausbildungszulagen besteht bei Ausbildungen, die in der\nAHV für den Anspruch auf Waisen- und Kinderrenten anerkannt sind.\nAls nachobligatorische Ausbildung gilt die Ausbildung, die auf die obliga-\ntorische Schulzeit folgt. Dauer und Ende der obligatorischen Schule richten\nsich nach den jeweiligen kantonalen Bestimmungen.\nAls Ausbildung gelten:\n• der Besuch von Schulen oder Kursen, die der Allgemeinbildung oder\nder Berufsbildung dienen;\n• die berufliche Ausbildung im Rahmen eines eigentlichen Lehrverhält-\nnisses, aber auch eine Tätigkeit ohne speziellen Berufsabschluss, die\neine systematische Vorbereitung auf eine zukünftige Erwerbstätigkeit\ndarstellt.\nNicht als in Ausbildung gilt, wer zur Hauptsache erwerbstätig ist und nur\nnebenbei eine Schule oder Kurse besucht.\nKein Anspruch auf Ausbildungszulagen besteht für Jugendliche, deren\njährliches Bruttoerwerbseinkommen 29 400 Franken übersteigt."
9,"Gibt es Besonderheiten, wenn ich Arbeitnehmerin oder\nArbeitnehmer bin?","Sie haben auch bei Teilzeitarbeit Anspruch auf die vollen Familienzulagen,\nsofern Ihr Lohn mindestens 612 Franken im Monat bzw. 7 350 Franken im\nJahr beträgt. Ist der Lohn geringer, so haben Sie Anspruch auf Familienzu-\nlagen für Nichterwerbstätige, sofern die Voraussetzungen erfüllt sind.\nWenn Sie bei verschiedenen Arbeitgebenden beschäftigt sind, werden die\nLöhne zusammengezählt. Zuständig für die Familienzulagen ist derjenige\nArbeitgebende, der den höchsten Lohn ausrichtet.\nIhr Anspruch auf Familienzulagen entsteht und erlischt mit dem Lohn-\nanspruch. Bei Arbeitsverhinderungen wie Krankheit oder Unfall werden\nIhnen die Familienzulagen auf jeden Fall während des Monats, in dem\ndie Arbeitsverhinderung eintritt, und während der drei darauf folgenden\nMonate ausgerichtet. Sie werden auch während verschiedener Urlaube\nausgerichtet, zum Beispiel während des Mutterschaftsurlaubs, längstens\njedoch während 16 Wochen, sowie während des Urlaubs des andern El-\nternteils, längstens jedoch während zwei Wochen."
31,Erhalten Kinder im Ausland Familienzulagen?,"Familienzulagen werden für Kinder im Ausland nur dann ausgerichtet,\nwenn die Schweiz aufgrund eines Abkommens über Soziale Sicherheit\ndazu verpflichtet ist:\n• An Staatsangehörige von EU- bzw. EFTA-Ländern werden Kinder-,\nAusbildungs- und Haushaltungszulagen nach dem FLG ausgerich-\ntet, wenn die Kinder in Ländern der EU- bzw. der EFTA wohnen. An\nStaatsangehörige von Belgien, Frankreich, Italien, Kroatien, Portugal,\nSlowenien und Spanien werden die Kinder- und Ausbildungszulagen\nweltweit exportiert.\n• An Staatsangehörige von Bosnien-Herzegowina, Montenegro, Nord-\nmazedonien, San Marino und der Türkei werden die Kinder- und Aus-\nbildungszulagen weltweit exportiert.\n• Das neue Abkommen mit dem Vereinigten Königreich, das seit dem\n1. November 2021 in Kraft ist, erstreckt sich nicht auf Famili-\nenleistungen; diese können

In [13]:
def highlight(s):

    return "\x1b[1;32m" + s + "\x1b[0m"

print("HELLO" + highlight("Hello, World!") + "BYBYE")

HELLOHello, World!BYBYE


In [ ]:
for i, row in eval_sample.iterrows():

    # RAG answer based on a selected RAG pipeline (loaded from config.yaml)
    rag_answer = None

    openai_eval_prompt = JUDGE_PROMPT.format(user_query=row.question, gold_answer=row.answer, rag_answer=rag_answer)
    openai_messages = [{"role": "system", "content": openai_eval_prompt},]

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=openai_messages
    )

    llm_judge_eval = response.choices[0].message.content

    # extract the rating


In [ ]:
# plot and save the results